In [ ]:
!pip install yt-dlp

In [ ]:
!pip install ffmpeg

In [ ]:
import yt_dlp
import random
import os
from datetime import datetime

In [ ]:
# Opzioni per yt-dlp
ydl_opts = {
    'quiet': True,  
    'ignoreerrors': True,  
    'extract_flat': False,  
    'skip_download': True,  
}

# Funzione per estrarre URL di video validi
def ottieni_video_ita_auto(playlist_url):
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        risultato = ydl.extract_info(playlist_url, download=False)
        video_validi = []
        if 'entries' in risultato:
            for entry in risultato['entries']:
                if entry is None:
                    continue
                controllo_data = entry.get('upload_date')
                if controllo_data and int(controllo_data[:4]) >= 2023 and entry.get('duration', 0) >= 900:
                    # Verifica sottotitoli forniti dall'autore e automatici in italiano
                    sottotitoli_ita = entry.get('subtitles', {}).get('it')
                    sottotitoli_ita_auto = entry.get('automatic_captions', {}).get('it')
                    if sottotitoli_ita and sottotitoli_ita_auto:  
                        video_url = entry.get('webpage_url')
                        if video_url:
                            video_validi.append(video_url)
        print(f"Video validi trovati: {len(video_validi)}")
        return video_validi

# URL della playlist
playlist_url = "https://www.youtube.com/playlist?list=PLBG7i6yKSeBxImaXXoJCK-Zi4sGSlHUrs"  

# Estrazione di URL validi
video_urls = ottieni_video_ita_auto(playlist_url)

if len(video_urls) >= 15:
    video_selezionati = random.sample(video_urls, 15)
    print("Elenco dei 15 video selezionati:")
    for url in video_selezionati:
        print(url)
else:
    print(f"Non ci sono abbastanza video validi. Video trovati: {len(video_urls)}.")
    print("Elenco dei video disponibili:")
    for url in video_urls:
        print(url)


In [ ]:
# Download sottotitoli forniti dall'autore per i video selezionati

cartella_sottotitoli_autore = './sottotitoli_autore'
os.makedirs(cartella_sottotitoli_autore, exist_ok=True)


ydl_opzioni = {
    'ffmpeg_location': r"C:\Users\Silvia\Desktop\ffmpeg-2024-10-24-git-153a6dc8fa-essentials_build\bin",  # da sostituire con il proprio percorso di ffmpeg
    'quiet': True,
    'ignoreerrors': True,
    'writesubtitles': True,      
    'subtitleslangs': ['it'],     
    'subtitlesformat': 'srt',     
    'skip_download': True,
    'outtmpl': os.path.join(cartella_sottotitoli_autore, '%(title)s [%(language)s].srt')
}


if len(video_selezionati) > 0:
    with yt_dlp.YoutubeDL(ydl_opzioni) as ydl:
        ydl.download(video_selezionati)
else:
    print("Nessun video disponibile con i criteri specificati.")

In [ ]:
import subprocess

In [ ]:
# Funzione per convertire i file .vtt in .srt nella cartella specificata, ignorando .ipynb_checkpoints
# Eliminazione file .vtt originale dopo la conversione

def converti_vtt_a_srt(cartella):
    for file_name in os.listdir(cartella):
        file_path = os.path.join(cartella, file_name)
        
        
        if os.path.isdir(file_path) and file_name == '.ipynb_checkpoints':
            continue
        
        if file_name.endswith('.vtt'):
            output_file = file_path.replace('.vtt', '.srt')
            
            comando = [
                'ffmpeg', '-i', file_path,
                output_file
            ]
            
            try:
                subprocess.run(comando, check=True)
                print(f"Convertito '{file_name}' in '{output_file}'.")
                
                os.remove(file_path)
                print(f"File originale '{file_name}' eliminato.")
                
                
            except subprocess.CalledProcessError as e:
                print(f"Errore nella conversione di '{file_name}': {e}")

cartella_da_processare = './sottotitoli_autore'

converti_vtt_a_srt(cartella_da_processare)



In [ ]:
import re

In [ ]:
!pip install youtube_transcript_api

In [ ]:
# Comando per trovare ID dei video selezionati

urls = [
    "https://www.youtube.com/watch?v=6ktrqFoy-e4&t=3s",
    "https://www.youtube.com/watch?v=BgZoWI6zTjc&t=14s",
    "https://www.youtube.com/watch?v=QIs9DCzvdIA&t=2s",
    "https://www.youtube.com/watch?v=aHR1hJBZslI&t=34s",
    "https://www.youtube.com/watch?v=AQCECeD1Qss",
    "https://www.youtube.com/watch?v=xMKnDtwLvVY&t=10s",
    "https://www.youtube.com/watch?v=6q4QJqFpjkk&t=790s",
    "https://www.youtube.com/watch?v=ZSMDJ-co-Xs",
    "https://www.youtube.com/watch?v=rX4gGScg2Hs",
    "https://www.youtube.com/watch?v=Xysy-6pQpsQ&t=45s",
    "https://www.youtube.com/watch?v=MTBlQJBUhd8",
    "https://www.youtube.com/watch?v=LWvOlZ4hPU0&t=14s",
    "https://www.youtube.com/watch?v=GzCB_4JzJIw&t=42s",
    "https://www.youtube.com/watch?v=opYDgs6eR3c",
    "https://www.youtube.com/watch?v=jtd_vwY5B8U"
]


video_id_pattern = re.compile(r"v=([a-zA-Z0-9_-]+)")

video_ids = []

for url in urls:
    match = video_id_pattern.search(url)
    if match:
        video_ids.append(match.group(1))

print(video_ids)


In [ ]:
import unicodedata

In [ ]:
# Funzione per scaricare i sottotitoli automatici in italiano data una lista di video ID
# Prima ottiene la lista delle trascrizioni disponibili e poi tra queste controlla se è disponibile la versione automatica in italiano
# Scarica la trascrizione e la salva in un file
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import SRTFormatter
from yt_dlp import YoutubeDL

video_ids = [
    '6ktrqFoy-e4', 'BgZoWI6zTjc', 'QIs9DCzvdIA', 'aHR1hJBZslI',
    'AQCECeD1Qss', 'xMKnDtwLvVY', '6q4QJqFpjkk', 'ZSMDJ-co-Xs',
    'rX4gGScg2Hs', 'Xysy-6pQpsQ', 'MTBlQJBUhd8', 'LWvOlZ4hPU0',
    'GzCB_4JzJIw', 'opYDgs6eR3c', 'jtd_vwY5B8U'
]

cartella_output = 'video_autosub'

os.makedirs(cartella_output, exist_ok=True)

# Rimuove gli accenti e normalizza il titolo
def normalizza_titolo(titolo):
    titolo_normalizzato = unicodedata.normalize('NFKD', titolo)
    titolo_normalizzato = ''.join([c for c in titolo_normalizzato if not unicodedata.combining(c)])
    titolo_normalizzato = re.sub(r'\W+', '-', titolo_normalizzato.lower())
    return titolo_normalizzato.strip('-')

def ottieni_titolo(video_id):
    try:
        with YoutubeDL({'quiet': True}) as ydl:
            info = ydl.extract_info(f'https://www.youtube.com/watch?v={video_id}', download=False)
            return info.get('title', f'video_{video_id}')
    except Exception as e:
        print(f"Impossibile ottenere il titolo per {video_id}: {e}")
        return f'video_{video_id}'

def download_sottotitoli_auto(video_id):
    try:
        trascrizioni = YouTubeTranscriptApi.list_transcripts(video_id)
        
        auto_trascrizioni = None
        for trascrizione in trascrizioni:
            if trascrizione.language_code == 'it' and trascrizione.is_generated:
                auto_trascrizioni = trascrizione
                break

        if auto_trascrizioni:
            dati_trascritti = auto_trascrizioni.fetch()
            formatter = SRTFormatter()
            srt_trascrizione = formatter.format_transcript(dati_trascritti)
            if video_id == 'aHR1hJBZslI':
                titolo_file = 'i-camionisti-immortali-del-pakistan'
            else:
                titolo = ottieni_titolo(video_id)
                titolo_file = normalizza_titolo(titolo)

            # Limita la lunghezza del nome del file a 45 caratteri
            # perchè file con nomi troppo lunghi potrebbero creare problematiche in punti seguenti del codice
            titolo_file = titolo_file[:45]

            output_path = os.path.join(cartella_output, f'{titolo_file}.srt')

            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(srt_trascrizione)

            print(f'Sottotitoli automatici per {video_id} scaricati con successo in italiano e salvati in {cartella_output}.')
        else:
            print(f'Nessun sottotitolo automatico disponibile in italiano per {video_id}.')
    except Exception as e:
        print(f'Impossibile scaricare i sottotitoli automatici per {video_id}: {e}')

for video_id in video_ids:
    download_sottotitoli_auto(video_id)


In [ ]:
# Funzione che pulisce i file scaricati con sottotitoli forniti dall'autore
# Regex per identificare e rimuovere le linee con timestamp e testo tra parentesi quadre
# Sovrascrittura del file originale con testo pulito

def pulisci_sottotitoli(cartella_input):
    
    timestamp_pattern = re.compile(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}')
    parentesi_quadre_pattern = re.compile(r'\[.*?\]')
    
    for file_name in os.listdir(cartella_input):
        file_path = os.path.join(cartella_input, file_name)

        if file_name.endswith('.srt'):
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                
            testo_pulito = []
            for line in lines:
                if not line.strip().isdigit() and not timestamp_pattern.match(line):  
                    line = parentesi_quadre_pattern.sub('', line)  
                    testo_pulito.append(line.strip())      

            with open(file_path, 'w', encoding='utf-8') as file:
                 file.write("\n".join(testo_pulito))
                
            print(f"Minutaggio e parentesi quadre rimossi dal file: {file_name}")

cartella_da_processare = 'sottotitoli_autore'

pulisci_sottotitoli(cartella_da_processare)


In [ ]:
# Funzione che pulisce i file scaricati con sottotitoli generati automaticamente da YouTube

input_folder = 'video_autosub'
output_folder = 'video_autosub_clean'

os.makedirs(output_folder, exist_ok=True)

timestamp_pattern = re.compile(r"^\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}$")
non_text_pattern = re.compile(r"^\[.*\]$") 

def pulizia_sottotitoli_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        linee = f.readlines()

    contenuto_pulito = []
    ultima_linea_vuota = True

    for linea in linee:
        pulizia_linea = linea.strip()

        if timestamp_pattern.match(pulizia_linea) or pulizia_linea.isdigit() or non_text_pattern.match(pulizia_linea):
            continue
        
        if pulizia_linea:
            contenuto_pulito.append(linea)
            ultima_linea_vuota = False
        
        elif not ultima_linea_vuota:
            contenuto_pulito.append('\n')
            ultima_linea_vuota = True

    return "".join(contenuto_pulito)

for filename in os.listdir(input_folder):
    if filename.endswith('.srt'):
        input_path = os.path.join(input_folder, filename)
        contenuto_pulito = pulizia_sottotitoli_file(input_path)

        output_path = os.path.join(output_folder, filename)
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(contenuto_pulito)

        print(f'File processato e salvato: {output_path}')


In [ ]:
from collections import Counter
import string

In [ ]:
# Funzione per rimuovere la punteggiatura da una parola

def pulisci_parole(parola):
    return parola.strip(string.punctuation)

# Funzione per analizzare le statistiche di un singolo file
def ottieni_statistiche_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as infile:
            text = infile.read()
    except Exception as e:
        print(f"Errore durante la lettura del file {file_path}: {e}")
        return

    parole = [pulisci_parole(parola) for parola in text.split() if pulisci_parole(parola)]
    n_parole = len(parole)
    n_uniche = len(set(parole))
        
    print(f"\nAnalisi del file: {file_path}")
    print("Numero totale di parole:", n_parole)
    print("Numero di parole uniche:", n_uniche)
    print("TTR (Type-Token Ratio):", n_uniche / n_parole)
    
    counter = Counter(parole)
    most_common_words = counter.most_common(10)
    
    print("Frequenze delle parole più comuni:")
    for parola, frequency in most_common_words:
        print("\t", parola, "(" + str(frequency) + ")")

# Funzione per iterare su tutti i file .srt nella cartella specificata
def analizza_statistiche_cartella (folder_path):
    if not os.path.isdir(folder_path):
        print(f"{folder_path} non è una cartella valida.")
        return

    if not any(filename.endswith('.srt') for filename in os.listdir(folder_path)):
        print(f"La cartella {folder_path} non contiene file .srt.")
        return

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        if os.path.isfile(file_path) and filename.endswith('.srt'):
            ottieni_statistiche_file(file_path) 

analizza_statistiche_cartella(r'sottotitoli_autore')


In [ ]:
analizza_statistiche_cartella(r'video_autosub_clean')

In [ ]:
pip install numpy==1.26.0 scipy==1.10.1 nltk==3.8.1

In [ ]:
import nltk
import unicodedata
from nltk.tokenize import word_tokenize

In [ ]:
# Funzioni per svolgere la tokenizzazione correttamente

nltk.download('punkt')

# Funzione per rimuovere caratteri di controllo invisibili
def rimuovi_controlli(token):
    return ''.join(c for c in token if unicodedata.category(c)[0] != 'C')

# Funzione per unire l'apostrofo al token precedente e separare i caratteri successivi
def separa_apostrofi(tokens):
    nuovi_tokens = []
    i = 0
    while i < len(tokens):
        if tokens[i] == "McDonald" and i + 1 < len(tokens) and tokens[i + 1] == "'s":
            nuovi_tokens.append("McDonald's")
            i += 2
        elif "'" in tokens[i] and tokens[i] != "'":
            parti = tokens[i].split("'")
            if len(parti) == 2 and parti[0] and parti[1]:
                nuovi_tokens.append(parti[0] + "'")
                nuovi_tokens.append(parti[1])
            else:
                nuovi_tokens.append(tokens[i])
            i += 1
        elif tokens[i] == "'" and i > 0:
            nuovi_tokens[-1] += "'"
            i += 1
        else:
            nuovi_tokens.append(tokens[i])
            i += 1
    return nuovi_tokens

# Funzione per separare i trattini iniziali come token singoli
def separa_trattini(tokens):
    nuovi_tokens = []
    for token in tokens:
        if token.startswith('-') and len(token) > 1 and not re.match(r'\b\w+-\w+\b', token):
            nuovi_tokens.append('-')
            nuovi_tokens.append(token[1:])
        else:
            nuovi_tokens.append(token)
    return nuovi_tokens

# Funzione per unire i token contenenti solo '’' al token precedente
def unisci_accento(tokens):
    nuovi_tokens = []
    for i, token in enumerate(tokens):
        if token == "’" and i > 0:  
            nuovi_tokens[-1] += token  
        else:
            nuovi_tokens.append(token)
    return nuovi_tokens

# Funzione per leggere e tokenizzare i file di sottotitoli
def tokenizza_sottotitoli(cartella_input, cartella_output):
    if not os.path.exists(cartella_input):
        print("La cartella di input specificata non esiste.")
        return
    else:
        print("La cartella di input esiste e il codice continua...")
    
    os.makedirs(cartella_output, exist_ok=True)
    files_nella_cartella = sorted(os.listdir(cartella_input))
    
    for file_name in files_nella_cartella:
        file_path = os.path.join(cartella_input, file_name)
        
        if file_name == '.ipynb_checkpoints':
            print(f"Ignorando la cartella: {file_name}")
            continue
        
        if os.path.isfile(file_path) and file_name.endswith('.srt'):
            print(f"Elaborando il file: {file_name}")
            
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    testo = file.read()
                
                # Passaggi di tokenizzazione e trasformazione
                tokens = word_tokenize(testo)
                tokens_apostrofati = separa_apostrofi(tokens)
                tokens_trattini_separati = separa_trattini(tokens_apostrofati)
                tokens_accento_unito = unisci_accento(tokens_trattini_separati)
                tokens_formattati = [f"[{rimuovi_controlli(token)}]" for token in tokens_accento_unito]
                testo_tokenizzato = " ".join(tokens_formattati)
                
                output_path = os.path.join(cartella_output, file_name.replace('.srt', '_tokenized.txt'))
                with open(output_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(testo_tokenizzato)
                
                print(f"File tokenizzato salvato in: {output_path}")
                
            except Exception as e:
                print(f"Errore nell'aprire il file '{file_name}': {e}")
        else:
            print(f"Il file '{file_name}' non è un file .srt valido o è una cartella.")

cartella_input = 'sottotitoli_autore'
cartella_output = 'tokenizza_sottotitoli_autore'
tokenizza_sottotitoli(cartella_input, cartella_output)


In [ ]:
# Funzione per la tokenizzazione che tiene conto di fenomeni quali l'apostrofo, il quale viene sempre aggiunto al token precedente
# Nella funzione è incluso il caso specifico di McDonald's che va tokenizzato come token unico
# I file risultanti sono in formato .txt non più .srt

nltk.download('punkt')

def rimuovi_controlli(token):
    return ''.join(c for c in token if unicodedata.category(c)[0] != 'C')

def separa_apostrofi(tokens):
    nuovi_tokens = []
    i = 0
    while i < len(tokens):
        
        if tokens[i] == "McDonald" and i + 1 < len(tokens) and tokens[i + 1] == "'s":
            nuovi_tokens.append("McDonald's")
            i += 2  
        
        elif "'" in tokens[i] and tokens[i] != "'":
            parti = tokens[i].split("'")
            if len(parti) == 2 and parti[0] and parti[1]:  
                nuovi_tokens.append(parti[0] + "'")
                nuovi_tokens.append(parti[1])
            else:
                nuovi_tokens.append(tokens[i])
            i += 1
        elif tokens[i] == "'" and i > 0:  
            nuovi_tokens[-1] += "'"  
            i += 1
        else:
            nuovi_tokens.append(tokens[i])
            i += 1
    return nuovi_tokens

def tokenizza_sottotitoli(cartella_input):
    if not os.path.exists(cartella_input):
        print("La cartella specificata non esiste.")
        return
    else:
        print("La cartella esiste e il codice continua...")
    
    cartella_output = 'sottotitoli_tokenizzati'
    os.makedirs(cartella_output, exist_ok=True)
    
    files_nella_cartella = os.listdir(cartella_input)
    
    for file_name in files_nella_cartella:
        file_path = os.path.join(cartella_input, file_name)
        
        if os.path.isfile(file_path) and file_name.endswith('.srt'):
            print(f"Elaborando il file: {file_name}")
            
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    testo = file.read()
                
                tokens = word_tokenize(testo)
                
                tokens_apostrofati = separa_apostrofi(tokens)
                
                tokens_formattati = [f"[{rimuovi_controlli(token)}]" for token in tokens_apostrofati]
                
                output_path = os.path.join(cartella_output, file_name.replace('.srt', '_tokenized.txt'))
                with open(output_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(' '.join(tokens_formattati))  # Salva i token formattati
                
                print(f"File tokenizzato salvato in: {output_path}")
                
            except Exception as e:
                print(f"Errore nell'aprire il file '{file_name}': {e}")
        else:
            print(f"Il file '{file_name}' non è un file .srt valido o è una cartella.")

tokenizza_sottotitoli('video_autosub_clean')


In [ ]:
!pip install stanza

In [ ]:
import stanza
stanza.download('it')

In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
print("Numpy version:", numpy.__version__)
print("Stanza version:", stanza.__version__)

In [ ]:
import sys
!{sys.executable} -m pip install numpy==2.0.2

In [ ]:
# Funzione che prende in input i file tokenizzati e ne esegue la lemmatizzazione

if not os.path.exists(os.path.expanduser('~/.stanza_resources/it')):
    stanza.download('it')

nlp = stanza.Pipeline('it', processors='tokenize,pos,lemma', tokenize_no_ssplit=True)

def lemmatizza_token(cartella_input):
    if not os.path.exists(cartella_input):
        print("La cartella specificata non esiste.")
        return
    else:
        print("La cartella esiste e il codice continua...")
        
    cartella_output = 'sottotitoli_lemmatizzati_autore'
    os.makedirs(cartella_output, exist_ok=True)
    
    for file_name in os.listdir(cartella_input):
        file_path = os.path.join(cartella_input, file_name)
        
        if os.path.isfile(file_path) and file_name.endswith('_tokenized.txt'):
            print(f"Elaborando il file: {file_name}")
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    tokens = file.read().split()

                tokens_puliti = [token.strip("[]") for token in tokens]

                testo = ' '.join(tokens_puliti)
                doc = nlp(testo)

                tokens_lemmatizzati = [f"[{word.lemma}]" for sentence in doc.sentences for word in sentence.words]

                output_path = os.path.join(cartella_output, file_name.replace('_tokenized.txt', '_lemmatized.txt'))
                with open(output_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(' '.join(tokens_lemmatizzati))
                print(f"File lemmatizzato salvato in: {output_path}")

            except Exception as e:
                print(f"Errore nell'elaborazione del file '{file_name}': {e}")
        else:
            print(f"Il file '{file_name}' non è un file tokenizzato valido o è una cartella.")


lemmatizza_token('tokenizza_sottotitoli_autore')

In [ ]:
# Funzione che prende in input i file tokenizzati e ne esegue la lemmatizzazione

if not os.path.exists(os.path.expanduser('~/.stanza_resources/it')):
    stanza.download('it')

nlp = stanza.Pipeline('it', processors='tokenize,pos,lemma', tokenize_no_ssplit=True)

def lemmatizza_token(cartella_input):
    if not os.path.exists(cartella_input):
        print("La cartella specificata non esiste.")
        return
    else:
        print("La cartella esiste e il codice continua...")
        
    cartella_output = 'sottotitoli_lemmatizzati_youtube'
    os.makedirs(cartella_output, exist_ok=True)
    
    for file_name in os.listdir(cartella_input):
        file_path = os.path.join(cartella_input, file_name)
        
        if os.path.isfile(file_path) and file_name.endswith('_tokenized.txt'):
            print(f"Elaborando il file: {file_name}")
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    tokens = file.read().split()

                tokens_puliti = [token.strip("[]") for token in tokens]

                testo = ' '.join(tokens_puliti)
                doc = nlp(testo)

                tokens_lemmatizzati = [f"[{word.lemma}]" for sentence in doc.sentences for word in sentence.words]

                output_path = os.path.join(cartella_output, file_name.replace('_tokenized.txt', '_lemmatized.txt'))
                with open(output_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(' '.join(tokens_lemmatizzati))
                print(f"File lemmatizzato salvato in: {output_path}")

            except Exception as e:
                print(f"Errore nell'elaborazione del file '{file_name}': {e}")
        else:
            print(f"Il file '{file_name}' non è un file tokenizzato valido o è una cartella.")


lemmatizza_token('sottotitoli_tokenizzati')

In [ ]:
# Funzione che prende in input i file tokenizzati e ne esegue il PoS tagging
# Stanza si basa sul Universal Dependencies (UD) tagset per i tag PoS

stanza.download('it') 
nlp = stanza.Pipeline('it', processors='tokenize,pos', tokenize_no_ssplit=True)

def pos_tagging(cartella_input):
    if not os.path.exists(cartella_input):
        print("La cartella specificata non esiste.")
        return
    else:
        print("La cartella esiste e il codice continua...")

    cartella_output = 'sottotitoli_pos_autore'
    os.makedirs(cartella_output, exist_ok=True)
    
    for file_name in os.listdir(cartella_input):
        file_path = os.path.join(cartella_input, file_name)
        
        if os.path.isfile(file_path) and file_name.endswith('_tokenized.txt'):
            print(f"Elaborando il file: {file_name}")
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    tokens = file.read().split()

                tokens_puliti = [token.strip("[]") for token in tokens]

                testo = ' '.join(tokens_puliti)
                doc = nlp(testo)

                tokens_pos_tagged = [f"[{word.text}/{word.upos}]" for sentence in doc.sentences for word in sentence.words]

                output_path = os.path.join(cartella_output, file_name.replace('_tokenized.txt', '_pos_tagged.txt'))
                with open(output_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(' '.join(tokens_pos_tagged))
                
                print(f"File con PoS tagging salvato in: {output_path}")
            
            except Exception as e:
                print(f"Errore nell'aprire il file '{file_name}': {e}")
        else:
            print(f"Il file '{file_name}' non è un file tokenizzato valido o è una cartella.")

pos_tagging('tokenizza_sottotitoli_autore')


In [ ]:
# Funzione che prende in input i file tokenizzati e ne esegue il PoS tagging
# Stanza si basa sul Universal Dependencies (UD) tagset per i tag PoS

stanza.download('it') 
nlp = stanza.Pipeline('it', processors='tokenize,pos', tokenize_no_ssplit=True)

def pos_tagging(cartella_input):
    if not os.path.exists(cartella_input):
        print("La cartella specificata non esiste.")
        return
    else:
        print("La cartella esiste e il codice continua...")

    cartella_output = 'sottotitoli_pos_youtube'
    os.makedirs(cartella_output, exist_ok=True)
    
    for file_name in os.listdir(cartella_input):
        file_path = os.path.join(cartella_input, file_name)
        
        if os.path.isfile(file_path) and file_name.endswith('_tokenized.txt'):
            print(f"Elaborando il file: {file_name}")
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    tokens = file.read().split()

                tokens_puliti = [token.strip("[]") for token in tokens]

                testo = ' '.join(tokens_puliti)
                doc = nlp(testo)

                tokens_pos_tagged = [f"[{word.text}/{word.upos}]" for sentence in doc.sentences for word in sentence.words]

                output_path = os.path.join(cartella_output, file_name.replace('_tokenized.txt', '_pos_tagged.txt'))
                with open(output_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(' '.join(tokens_pos_tagged))
                
                print(f"File con PoS tagging salvato in: {output_path}")
            
            except Exception as e:
                print(f"Errore nell'aprire il file '{file_name}': {e}")
        else:
            print(f"Il file '{file_name}' non è un file tokenizzato valido o è una cartella.")

pos_tagging('sottotitoli_tokenizzati')

In [ ]:
# INIZIO CONFRONTO

In [ ]:
# Funzione per verificare i file nelle due cartelle

def verifica_file(cartella):
    if not os.path.exists(cartella):
        print(f"La cartella '{cartella}' non esiste!")
        return
    print(f"\nFile nella cartella '{cartella}':")
    for file in os.listdir(cartella):
        print(f"- {file}")


verifica_file("tokenizza_sottotitoli_autore")
verifica_file("sottotitoli_tokenizzati")

In [ ]:
import difflib
import hashlib
from nltk import download

In [ ]:
!python -m venv nuovo_env
!nuovo_env\Scripts\activate

In [ ]:
!pip install numpy==1.24.4 scipy==1.10.1

In [ ]:
!pip show numpy
!pip show scipy

In [ ]:
import gensim
import sklearn
import statsmodels
import numba
import skimage

print("gensim:", gensim.__version__)
print("scikit-learn:", sklearn.__version__)
print("statsmodels:", statsmodels.__version__)
print("numba:", numba.__version__)
print("scikit-image:", skimage.__version__)

In [ ]:
!pip install plotly
!pip show plotly

In [ ]:
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# Assicurarsi che le due cartelle abbiano lo stesso numero di file 
# Confronto tokenizzazione tra cartelle

cartella_autore = "tokenizza_sottotitoli_autore"
cartella_youtube = "sottotitoli_tokenizzati"

file_autore = sorted(os.listdir(cartella_autore))
file_youtube = sorted(os.listdir(cartella_youtube))


if len(file_autore) != len(file_youtube):
    raise ValueError("Le due cartelle contengono un numero diverso di file.")


risultati_confronto = []

for file_a, file_y in zip(file_autore, file_youtube):
    
    path_a = os.path.join(cartella_autore, file_a)
    path_y = os.path.join(cartella_youtube, file_y)

    with open(path_a, "r", encoding="utf-8") as fa, open(path_y, "r", encoding="utf-8") as fc:
        tokens_a = set(fa.read().split())  
        tokens_y = set(fc.read().split())  

    # Individuazione delle differenze
    solo_in_autore = tokens_a - tokens_y
    solo_in_youtube = tokens_y - tokens_a

    confronto = {
        "File Autore": file_a,
        "File YouTube": file_y,
        "Solo in Autore (N. Token)": len(solo_in_autore),
        "Solo in YouTube (N. Token)": len(solo_in_youtube),
        "Token Solo in Autore": ", ".join(sorted(solo_in_autore)) if solo_in_autore else "Nessuno",
        "Token Solo in YouTube": ", ".join(sorted(solo_in_youtube)) if solo_in_youtube else "Nessuno",
    }
    risultati_confronto.append(confronto)

# Creazione DataFrame e uso di Plotly per creazione di una tabella interattiva basata sul DataFrame
df_confronto = pd.DataFrame(risultati_confronto)


fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_confronto.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_confronto[col].tolist() for col in df_confronto.columns],
               fill_color='lavender',
               align='left'))
])

fig.show()

df_confronto.to_csv("confronto_tokenizzazione_interattivo.csv", index=False, encoding="utf-8")

print("La tabella del confronto è stata salvata in 'confronto_tokenizzazione_interattivo.csv'.")

In [ ]:
# Confronto lemmatizzazione

cartella_autore = "sottotitoli_lemmatizzati_autore"
cartella_youtube = "sottotitoli_lemmatizzati_youtube"

file_autore = sorted(os.listdir(cartella_autore))
file_youtube = sorted(os.listdir(cartella_youtube))


risultati_confronto = []

for file_a, file_y in zip(file_autore, file_youtube):
    
    path_a = os.path.join(cartella_autore, file_a)
    path_y = os.path.join(cartella_youtube, file_y)


    with open(path_a, "r", encoding="utf-8") as fa, open(path_y, "r", encoding="utf-8") as fy:
        lemmi_a = set(lemma.lower() for lemma in fa.read().split())  
        lemmi_y = set(lemma.lower() for lemma in fy.read().split())  

    solo_in_autore = lemmi_a - lemmi_y
    solo_in_youtube = lemmi_y - lemmi_a


    confronto = {
        "File Autore": file_a,
        "File YouTube": file_y,
        "Solo in Autore (N. Lemmi)": len(solo_in_autore),
        "Solo in YouTube (N. Lemmi)": len(solo_in_youtube),
        "Lemmi Solo in Autore": ", ".join(sorted(solo_in_autore)) if solo_in_autore else "Nessuno",
        "Lemmi Solo in YouTube": ", ".join(sorted(solo_in_youtube)) if solo_in_youtube else "Nessuno",
    }
    risultati_confronto.append(confronto)

df_confronto = pd.DataFrame(risultati_confronto)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_confronto.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_confronto[col].tolist() for col in df_confronto.columns],
               fill_color='lavender',
               align='left'))
])

fig.show()

df_confronto.to_csv("confronto_lemmatizzazioni_interattivo.csv", index=False, encoding="utf-8")

print("La tabella del confronto è stata salvata in 'confronto_lemmatizzazioni_interattivo.csv'.")

In [ ]:
# Confronto PoS Tagging 

cartella_autore = "sottotitoli_pos_autore"
cartella_youtube = "sottotitoli_pos_youtube"

file_autore = sorted(os.listdir(cartella_autore))
file_youtube = sorted(os.listdir(cartella_youtube))


risultati_confronto = []

for file_a, file_y in zip(file_autore, file_youtube):
    path_a = os.path.join(cartella_autore, file_a)
    path_y = os.path.join(cartella_youtube, file_y)

    with open(path_a, "r", encoding="utf-8") as fa, open(path_y, "r", encoding="utf-8") as fy:
        pos_a = set(pos.lower() for pos in fa.read().split())  
        pos_y = set(pos.lower() for pos in fy.read().split())  

    solo_in_autore = pos_a - pos_y
    solo_in_youtube = pos_y - pos_a

    confronto = {
        "File Autore": file_a,
        "File YouTube": file_y,
        "Solo in Autore (N. Pos)": len(solo_in_autore),
        "Solo in YouTube (N. Pos)": len(solo_in_youtube),
        "Pos Solo in Autore": ", ".join(sorted(solo_in_autore)) if solo_in_autore else "Nessuno",
        "Pos Solo in YouTube": ", ".join(sorted(solo_in_youtube)) if solo_in_youtube else "Nessuno",
    }
    risultati_confronto.append(confronto)


df_confronto = pd.DataFrame(risultati_confronto)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_confronto.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_confronto[col].tolist() for col in df_confronto.columns],
               fill_color='lavender',
               align='left'))
])

fig.show()

df_confronto.to_csv("confronto_pos_interattivo.csv", index=False, encoding="utf-8")

print("La tabella del confronto è stata salvata in 'confronto_pos_interattivo.csv'.")

In [ ]:
# Funzione per calcolare e visualizzare le percentuali di ogni PoS tag per ogni file

cartella_pos_autore = "sottotitoli_pos_autore"
cartella_pos_youtube = "sottotitoli_pos_youtube"

if not os.path.exists(cartella_pos_autore):
    raise FileNotFoundError("La cartella 'sottotitoli_pos_autore' non esiste.")
if not os.path.exists(cartella_pos_youtube):
    raise FileNotFoundError("La cartella 'sottotitoli_pos_youtube' non esiste.")

def calcola_frequenze_pos(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        tokens_pos = []
        for token in f.read().split():
            if "/" in token:
                pos = token.strip("[]").rsplit("/", 1)[-1] 
                tokens_pos.append(pos)
            else:
                print(f"Token non conforme ignorato: {token}")  
        conteggi_pos = Counter(tokens_pos)
        totale_token = sum(conteggi_pos.values())
        return {pos: (conteggio / totale_token) * 100 for pos, conteggio in conteggi_pos.items()}

file_autore = sorted(os.listdir(cartella_pos_autore))
file_youtube = sorted(os.listdir(cartella_pos_youtube))


risultati = []

for file_a, file_y in zip(file_autore, file_youtube):
    path_a = os.path.join(cartella_pos_autore, file_a)
    path_y = os.path.join(cartella_pos_youtube, file_y)

    frequenze_autore = calcola_frequenze_pos(path_a)
    frequenze_youtube = calcola_frequenze_pos(path_y)

    tutti_pos = set(frequenze_autore.keys()).union(set(frequenze_youtube.keys()))
    for pos in tutti_pos:
        risultati.append({
            "Nome File Autore": file_a,
            "Nome File YouTube": file_y,
            "PoS": pos,
            "Percentuale Autore": round(frequenze_autore.get(pos, 0), 2),
            "Percentuale YouTube": round(frequenze_youtube.get(pos, 0), 2)
        })

df_frequenze = pd.DataFrame(risultati)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_frequenze.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_frequenze[col].tolist() for col in df_frequenze.columns],
               fill_color='lavender',
               align='left'))
])

fig.show()

In [ ]:
# Funzione per visualizzare la percentuale complessiva dei PoS tag nei file 

cartella_pos_autore = "sottotitoli_pos_autore"
cartella_pos_youtube = "sottotitoli_pos_youtube"


if not os.path.exists(cartella_pos_autore):
    raise FileNotFoundError("La cartella 'sottotitoli_pos_autore' non esiste.")
if not os.path.exists(cartella_pos_youtube):
    raise FileNotFoundError("La cartella 'sottotitoli_pos_youtube' non esiste.")

def calcola_frequenze_pos(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        tokens_pos = []
        for token in f.read().split():
            if "/" in token:
                pos = token.strip("[]").rsplit("/", 1)[-1]  
                tokens_pos.append(pos)
        conteggi_pos = Counter(tokens_pos)
        totale_token = sum(conteggi_pos.values())
        return {pos: (conteggio / totale_token) * 100 for pos, conteggio in conteggi_pos.items()}

file_autore = sorted(os.listdir(cartella_pos_autore))
file_youtube = sorted(os.listdir(cartella_pos_youtube))

frequenze_totali_autore = Counter()
frequenze_totali_youtube = Counter()
totale_file = len(file_autore)

for file_a, file_y in zip(file_autore, file_youtube):
    path_a = os.path.join(cartella_pos_autore, file_a)
    path_y = os.path.join(cartella_pos_youtube, file_y)

    frequenze_autore = calcola_frequenze_pos(path_a)
    frequenze_youtube = calcola_frequenze_pos(path_y)

    for pos, perc in frequenze_autore.items():
        frequenze_totali_autore[pos] += perc
    for pos, perc in frequenze_youtube.items():
        frequenze_totali_youtube[pos] += perc

percentuali_medie_autore = {pos: round(total / totale_file, 2) for pos, total in frequenze_totali_autore.items()}
percentuali_medie_youtube = {pos: round(total / totale_file, 2) for pos, total in frequenze_totali_youtube.items()}

tutti_pos = set(percentuali_medie_autore.keys()).union(set(percentuali_medie_youtube.keys()))

risultati = []
for pos in tutti_pos:
    risultati.append({
        "PoS": pos,
        "Percentuale PoS autore": percentuali_medie_autore.get(pos, 0),
        "Percentuale PoS youTube": percentuali_medie_youtube.get(pos, 0)
    })

df_risultati = pd.DataFrame(risultati)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_risultati.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_risultati[col].tolist() for col in df_risultati.columns],
               fill_color='lavender',
               align='left'))
])


fig.show()